In [7]:
import pandas as pd

dir_suffix = "data/"
dir_suffix_entpackt = dir_suffix + "Daten_ausgepackt/"

pd.read_csv(dir_suffix + "tip25W_testdaten_template.csv")
df_prio = pd.read_csv(dir_suffix_entpackt + "order_products__prior.csv")
df_train = pd.read_csv(dir_suffix_entpackt + "order_products__train.csv")
df_products = pd.read_csv(dir_suffix_entpackt+"products.csv")

df_all = pd.concat([df_prio, df_train])

df_departments = pd.read_csv(dir_suffix_entpackt + "departments.csv")
df_departments = df_departments[df_departments["department_id"].between(1,5)]
df_departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


Aufgabe 1:
*Berechnen Sie die 5x5 Matrix M, die an Position m i,k jeweils angibt wie wahrscheinlich es ist, dass die
nachfolgende Bestellung eines Users mindestens einen Artikel aus Department k enthält
(, unter der Bedingung, dass die vorherige Bestellung desselben Users mindestens einen Artikel aus
dem Department mit ID i enthalten hat. (
Um die Matrix überschaubar zu halten, beschränkt sich die Aufgabe auf die
Departments 1 bis 5. Die wahren Wahrscheinlichkeiten sind natürlich
unbekannt; sie müssen also nur einen sinnvollen Schätzwert berechnen. Bei
Matrizen wird der erste Index nach unten, der zweite nach rechts
dargestellt.*

Also:
Wenn ein User etwas aus Department i gekauft hat: Wie wahrscheinlich ist es dann, dass er in seiner nächsten Bestellung etwas aus Department k kauft?

m i,k = P(nächste Bestellung enthält Department k | vorherige Bestellung enthielt Department i)

diese Bestellung aus k ∩ letze Bestellung aus k
_______________________________________________
letzte Bestellung aus k


Beispiel:

Für i = 3 und k = 1

Für alle user die in Prior einen Artikel aus 3 hatten und in der jetzigen Bestellung etwas aus 1??

P(A) = jemand hat aus 3 bestellt 
P(B) = jemand hat aus 1 bestellt

P(A ∩ B) = jemand hat prior aus 3 und jetzt aus 1 bestellt


| i, k    | 1 | 2 | 3 | 4 | 5 |
| -------- | ------- | ------- | ------- | ------- | ------- |
| 1        |   |   |   |   |   |
| 2        |   |   |   |   |   |
| 3        |   |   |   |   |   |
| 4        |   |   |   |   |   |
| 5        |   |   |   |   |   |

In [8]:
import pandas as pd
overview = pd.merge(df_all[:100][['order_id', 'product_id']], df_products[['product_id', 'department_id']], on="product_id", how="left")
overview.head(5)

,order_id,product_id,department_id
0,2,33120,16
1,2,28985,4
2,2,9327,13
3,2,45918,13
4,2,30035,13


In [9]:
from IPython.display import display

print("DF order Prio: ")
display(df_prio.head(1))

print("DF order train: ")
display(df_train.head(1))

print("DF orders:")
df_orders = pd.read_csv(dir_suffix_entpackt+"orders.csv")
display(df_orders.head(11))

print("DF products:")
display(df_products.head(1))

print("Departments:")
df_dep = pd.read_csv(dir_suffix_entpackt+"departments.csv")
display(df_dep.head(1))

DF order Prio: 


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1


DF order train: 


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1


DF orders:


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


DF products:


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19


Departments:


,department_id,department
0,1,frozen


In [10]:
# Sort so latest orders are first per user
df_sorted = df_orders.sort_values(['user_id', 'order_number'], ascending=[True, False])

latest_two_per_user = df_sorted.groupby('user_id').head(2).reset_index(drop=True)
latest_two_per_user.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1187899,1,train,11,4,8,14.0
1,2550362,1,prior,10,4,8,30.0
2,1492625,2,train,15,1,11,30.0
3,839880,2,prior,14,3,10,13.0
4,2774568,3,test,13,5,15,11.0
5,1402502,3,prior,12,1,15,15.0
6,329954,4,test,6,3,12,30.0
7,2557754,4,prior,5,5,13,0.0
8,2196797,5,train,5,0,11,6.0
9,157374,5,prior,4,1,18,19.0


In [5]:
latest_two_per_user_small = latest_two_per_user[["user_id", "order_id", "eval_set", "order_number"]]
latest_two_per_user_small

df_all_small = df_all[["product_id", "order_id"]]
df_all_small

,product_id,order_id
0,33120,2
1,28985,2
2,9327,2
3,45918,2
4,30035,2
...,...,...
1384612,14233,3421063
1384613,35548,3421063
1384614,35951,3421070
1384615,16953,3421070


In [60]:
orders_merged = pd.merge(latest_two_per_user_small, df_all_small, on="order_id",how="left")
orders_merged.head(30)

department_resolved = pd.merge(orders_merged[["user_id", "order_id","product_id","eval_set"]],df_products[["department_id", "product_id"]], on="product_id", how="left")

department_resolved.drop(labels=["product_id"], inplace=True, axis=1)
department_resolved = department_resolved[department_resolved["department_id"].between(1,5)]
department_resolved = department_resolved.reset_index(drop=1)
department_resolved.head(30)


,user_id,order_id,eval_set,department_id
0,2,1492625,train,1.0
1,2,1492625,train,4.0
2,2,1492625,train,4.0
3,2,1492625,train,4.0
4,2,1492625,train,4.0
5,2,1492625,train,4.0
6,2,1492625,train,4.0
7,2,1492625,train,4.0
8,2,1492625,train,1.0
9,2,1492625,train,1.0


In [ ]:
""" 
P(A) = jemand hat aus 3 bestellt 
P(B) = jemand hat aus 1 bestellt

P(A ∩ B) = jemand hat prior aus 3 und jetzt aus 1 bestellt 
"""

crossA = pd.crosstab(index=department_resolved["user_id"], columns=[department_resolved["department_id"],department_resolved["eval_set"]],margins=1)



department_id     1.0           2.0          3.0            4.0           5.0  \
eval_set        prior   train prior train  prior  train   prior   train prior   
user_id                                                                         
2                   1      11     0     0      0      0       4       7     0   
3                   0       0     0     0      0      0       2       0     0   
5                   0       0     0     0      0      0       5       4     0   
6                   0       0     0     0      0      0       1       0     0   
7                   0       0     0     0      0      0       4       1     0   
...               ...     ...   ...   ...    ...    ...     ...     ...   ...   
206205              0       1     0     0      1      0       3       7     0   
206207              1       0     0     0      0      0       2       0     0   
206208              0       0     0     0      1      0       8       0     0   
206209              0       1     0     0      0      1       1       1     0   
All            155990  100426  2720  1795  75299  48394  624649  409087  9749   

department_id            All  
eval_set      train           
user_id                       
2                 0       23  
3                 0        2  
5                 0        9  
6                 0        1  
7                 0        5  
...             ...      ...  
206205            0       12  
206207            0        3  
206208            0        9  
206209            0        4  
All            5598  1433707  

[185258 rows x 11 columns]

In [ ]:
prev = (
    df[department_resolved['order_type'] == 'prior']
    [['user_id', 'department_id']]
    .drop_duplicates()
    .rename(columns={'department_id': 'prev_dept'})
)

latest = (
    df[df['order_type'] == 'latest']
    [['user_id', 'department_id']]
    .drop_duplicates()
    .rename(columns={'department_id': 'latest_dept'})
)
